In [1]:
# LAB 2.4: Basic Data Saving
# Aim: Learn how to export cleaned or transformed datasets to various file formats.

# Objectives:
# Save a DataFrame to a .csv file using DataFrame.to_csv().
# Write Excel files using DataFrame.to_excel(), including to specific sheets.
# Export structured data to JSON using DataFrame.to_json() with appropriate orientation.
# Save to and read from Parquet files using DataFrame.to_parquet() (optional: install pyarrow or fastparquet).
# Use if_exists and index parameters to control overwriting and metadata saving.
# Practice saving intermediate processing outputs during a data analysis pipeline.

import pandas as pd
import numpy as np
import json
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

# Create output directory for saved files
output_dir = "saved_datasets"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"Created directory: {output_dir}")

print("\nLOADING DATASET")

# Try to load dataset from datasets folder, otherwise create sample data
try:
    # Try to load from datasets folder - you can specify your actual dataset here
    dataset_files = []
    if os.path.exists('datasets'):
        dataset_files = [f for f in os.listdir('datasets') if f.endswith(('.csv', '.xlsx', '.json'))]
        print(f"Available datasets: {dataset_files}")
    
    # If you have a specific dataset, load it here:
    # df = pd.read_csv('datasets/your_dataset.csv')
    
    # For demonstration, create sample Social Network Ads data similar to the lab
    print("Creating sample Social Network Ads dataset for demonstration...")
    
    np.random.seed(42)
    n_samples = 400
    
    # Create realistic Social Network Ads data
    df = pd.DataFrame({
        'User ID': range(15624807, 15624807 + n_samples),
        'Gender': np.random.choice(['Male', 'Female'], n_samples, p=[0.52, 0.48]),
        'Age': np.random.randint(18, 60, n_samples),
        'EstimatedSalary': np.random.normal(69000, 25000, n_samples).astype(int),
        'Purchased': np.random.choice([0, 1], n_samples, p=[0.64, 0.36])
    })
    
    # Add some data quality issues for cleaning demonstration
    # Add duplicates
    df = pd.concat([df, df.iloc[:5]], ignore_index=True)
    
    # Add some missing values
    df.loc[np.random.choice(df.index, 10, replace=False), 'EstimatedSalary'] = np.nan
    
    # Add some outliers
    df.loc[np.random.choice(df.index, 3, replace=False), 'Age'] = np.random.choice([100, 150, 200])
    
    print("Sample dataset created successfully!")
    print(f"Dataset shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")
    
except Exception as e:
    print(f"Error loading dataset: {e}")
    print("Please check your datasets folder or modify the loading code above.")

# Display basic information about the dataset
print("\nDataset Overview:")
print(df.head())
print(f"\nDataset Info:")
df.info()

print("\nBASIC DATA SAVING OPERATIONS")

# STEP 1: Save a DataFrame to a CSV file
print("\nSTEP 1: Save DataFrame to CSV file")

csv_filename = os.path.join(output_dir, "New_Social_Network_Ads_Data.csv")
df.to_csv(csv_filename, index=False)
print(f"Saved to CSV: {csv_filename}")

# Verify the saved file
if os.path.exists(csv_filename):
    file_size = os.path.getsize(csv_filename)
    print(f"File size: {file_size} bytes")
    
    # Read back and display first few rows
    df_csv_check = pd.read_csv(csv_filename)
    print(f"Verification - Shape after reading: {df_csv_check.shape}")
    print(f"First 3 rows:")
    print(df_csv_check.head(3))

# STEP 2: Save a DataFrame to Excel
print("\n\nSTEP 2: Save DataFrame to Excel")

try:
    excel_filename = os.path.join(output_dir, "New_Social_Network_Ads_Data.xlsx")
    df.to_excel(excel_filename, sheet_name="AdsData", index=False)
    print(f"Saved to Excel: {excel_filename}")
    
    # Verify the saved file
    if os.path.exists(excel_filename):
        file_size = os.path.getsize(excel_filename)
        print(f"File size: {file_size} bytes")
        
        # Read back and verify
        df_excel_check = pd.read_excel(excel_filename, sheet_name="AdsData")
        print(f"Verification - Shape after reading: {df_excel_check.shape}")
        
except ImportError:
    print("Excel export requires openpyxl or xlsxwriter. Install with:")
    print("pip install openpyxl")
    print("or")
    print("pip install xlsxwriter")

# STEP 3: Write to multiple Excel sheets
print("\n\nSTEP 3: Write to multiple Excel sheets")

try:
    multi_excel_filename = os.path.join(output_dir, "Multiple_Social_Network_Ads_Data.xlsx")
    
    # Create different views of the data for different sheets
    df_male = df[df['Gender'] == 'Male'].copy()
    df_female = df[df['Gender'] == 'Female'].copy()
    
    with pd.ExcelWriter(multi_excel_filename, engine='openpyxl') as writer:
        df.to_excel(writer, sheet_name="AllData", index=False)
        df_male.to_excel(writer, sheet_name="MaleUsers", index=False)
        df_female.to_excel(writer, sheet_name="FemaleUsers", index=False)
        
        # Create a summary sheet
        summary_data = {
            'Metric': ['Total Users', 'Male Users', 'Female Users', 'Average Age', 'Average Salary', 'Purchase Rate'],
            'Value': [
                len(df),
                len(df_male),
                len(df_female),
                f"{df['Age'].mean():.1f}",
                f"${df['EstimatedSalary'].mean():.0f}",
                f"{df['Purchased'].mean():.2%}"
            ]
        }
        summary_df = pd.DataFrame(summary_data)
        summary_df.to_excel(writer, sheet_name="Summary", index=False)
    
    print(f"Saved multi-sheet Excel: {multi_excel_filename}")
    print(f"Sheets created: AllData, MaleUsers, FemaleUsers, Summary")
    
    # Verify sheets
    xl_file = pd.ExcelFile(multi_excel_filename)
    print(f"Sheet names: {xl_file.sheet_names}")
    
except ImportError:
    print("Excel export requires openpyxl. Install with: pip install openpyxl")
except Exception as e:
    print(f"Error creating multi-sheet Excel: {e}")

# STEP 4: Save a DataFrame to JSON
print("\n\nSTEP 4: Save DataFrame to JSON")

# Different JSON orientations
json_orientations = ['records', 'index', 'values', 'columns']

for orient in json_orientations:
    json_filename = os.path.join(output_dir, f"Social_Network_Ads_Data_{orient}.json")
    
    if orient == 'records':
        # For records orientation, also demonstrate lines=True
        df.to_json(json_filename, orient=orient, lines=True)
        print(f"Saved JSON ({orient}, lines=True): {json_filename}")
    else:
        df.to_json(json_filename, orient=orient, indent=2)
        print(f"Saved JSON ({orient}): {json_filename}")
    
    # Show file size
    if os.path.exists(json_filename):
        file_size = os.path.getsize(json_filename)
        print(f"File size: {file_size} bytes")

# Demonstrate reading JSON back
print("\nReading JSON files back:")
for orient in json_orientations:
    json_filename = os.path.join(output_dir, f"Social_Network_Ads_Data_{orient}.json")
    try:
        if orient == 'records':
            df_json_check = pd.read_json(json_filename, lines=True)
        else:
            df_json_check = pd.read_json(json_filename, orient=orient)
        print(f"{orient}: Shape {df_json_check.shape}")
    except Exception as e:
        print(f"{orient}: Error - {e}")

# STEP 5: Save to Parquet (if available)
print("\n\nSTEP 5: Save DataFrame to Parquet (Optional)")

try:
    parquet_filename = os.path.join(output_dir, "Social_Network_Ads_Data.parquet")
    df.to_parquet(parquet_filename, index=False)
    print(f"Saved to Parquet: {parquet_filename}")
    
    # Verify file
    if os.path.exists(parquet_filename):
        file_size = os.path.getsize(parquet_filename)
        print(f"File size: {file_size} bytes")
        
        # Read back and verify
        df_parquet_check = pd.read_parquet(parquet_filename)
        print(f"Verification - Shape after reading: {df_parquet_check.shape}")
        
except ImportError:
    print("Parquet export requires pyarrow or fastparquet. Install with:")
    print("pip install pyarrow")
    print("or")
    print("pip install fastparquet")
except Exception as e:
    print(f"Error saving Parquet: {e}")

print("\nINDEX AND PARAMETER CONTROL")

# STEP 6: Save CSV with and without index
print("\nSTEP 6: Save CSV with different index settings")

# With index
indexed_filename = os.path.join(output_dir, "Indexed_Social_Network_Ads_Data.csv")
df.to_csv(indexed_filename, index=True)
print(f"Saved WITH index: {indexed_filename}")

# Without index
no_index_filename = os.path.join(output_dir, "NoIndex_Social_Network_Ads_Data.csv")
df.to_csv(no_index_filename, index=False)
print(f"Saved WITHOUT index: {no_index_filename}")

# Compare file sizes
indexed_size = os.path.getsize(indexed_filename)
no_index_size = os.path.getsize(no_index_filename)
print(f"With index file size: {indexed_size} bytes")
print(f"Without index file size: {no_index_size} bytes")
print(f"Difference: {indexed_size - no_index_size} bytes")

# Show the difference in content
print("\nContent comparison (first 3 lines):")
print("WITH index:")
with open(indexed_filename, 'r') as f:
    for i, line in enumerate(f):
        if i < 3:
            print(f"{line.strip()}")
        else:
            break

print("WITHOUT index:")
with open(no_index_filename, 'r') as f:
    for i, line in enumerate(f):
        if i < 3:
            print(f"{line.strip()}")
        else:
            break

print("\nDATA PROCESSING PIPELINE WITH INTERMEDIATE SAVES")

# STEP 7: Saving Intermediate Outputs
print("\nSTEP 7: Data processing pipeline with intermediate saves")

print(f"Original dataset shape: {df.shape}")
print(f"Original duplicates: {df.duplicated().sum()}")
print(f"Original missing values: {df.isnull().sum().sum()}")

# Step 1: Remove duplicates
print("\n1. Removing duplicates...")
df_cleaned = df.drop_duplicates()
cleaned_filename = os.path.join(output_dir, "Cleaned_Social_Network_Ads_Data.csv")
df_cleaned.to_csv(cleaned_filename, index=False)
print(f"Removed {len(df) - len(df_cleaned)} duplicates")
print(f"Saved cleaned data: {cleaned_filename}")
print(f"New shape: {df_cleaned.shape}")

# Step 2: Handle missing values
print("\n2. Handling missing values...")
df_no_missing = df_cleaned.dropna()
no_missing_filename = os.path.join(output_dir, "NoMissing_Social_Network_Ads_Data.csv")
df_no_missing.to_csv(no_missing_filename, index=False)
print(f"Removed {len(df_cleaned) - len(df_no_missing)} rows with missing values")
print(f"Saved data without missing values: {no_missing_filename}")
print(f"New shape: {df_no_missing.shape}")

# Step 3: Filter data (Age > 40)
print("\n3. Filtering data (Age > 40)...")
df_transformed = df_no_missing[df_no_missing['Age'] > 40].copy()
filtered_filename = os.path.join(output_dir, "Filtered_Social_Network_Ads_Data.xlsx")

try:
    df_transformed.to_excel(filtered_filename, index=False)
    print(f"Filtered to {len(df_transformed)} records (Age > 40)")
    print(f"Saved filtered data: {filtered_filename}")
    print(f"Final shape: {df_transformed.shape}")
except ImportError:
    # If Excel is not available, save as CSV
    filtered_csv_filename = os.path.join(output_dir, "Filtered_Social_Network_Ads_Data.csv")
    df_transformed.to_csv(filtered_csv_filename, index=False)
    print(f"Filtered to {len(df_transformed)} records (Age > 40)")
    print(f"Saved filtered data (CSV): {filtered_csv_filename}")
    print(f"Final shape: {df_transformed.shape}")

# Step 4: Create age groups and save
print("\n4. Creating age groups analysis...")
df_transformed['AgeGroup'] = pd.cut(df_transformed['Age'], 
                                   bins=[0, 30, 40, 50, 100], 
                                   labels=['<30', '30-40', '40-50', '>50'])

age_group_summary = df_transformed.groupby('AgeGroup').agg({
    'User ID': 'count',
    'EstimatedSalary': 'mean',
    'Purchased': 'mean'
}).round(2)

age_group_summary.columns = ['User_Count', 'Avg_Salary', 'Purchase_Rate']
age_summary_filename = os.path.join(output_dir, "Age_Group_Analysis.csv")
age_group_summary.to_csv(age_summary_filename)
print(f"Created age group analysis: {age_summary_filename}")
print("Age Group Summary:")
print(age_group_summary)

print("\nADVANCED SAVING OPTIONS")

# Demonstrate various CSV parameters
print("\nAdvanced CSV saving options:")

# Custom separator
custom_sep_filename = os.path.join(output_dir, "Custom_Separator_Data.txt")
df_transformed.to_csv(custom_sep_filename, sep='|', index=False)
print(f"Saved with custom separator (|): {custom_sep_filename}")

# Only specific columns
selected_cols_filename = os.path.join(output_dir, "Selected_Columns_Data.csv")
df_transformed[['User ID', 'Age', 'Purchased']].to_csv(selected_cols_filename, index=False)
print(f"Saved selected columns only: {selected_cols_filename}")

# With custom headers
custom_header_filename = os.path.join(output_dir, "Custom_Headers_Data.csv")
df_transformed.to_csv(custom_header_filename, 
                     columns=['User ID', 'Gender', 'Age', 'EstimatedSalary', 'Purchased'],
                     header=['ID', 'Sex', 'Years', 'Income', 'Bought'],
                     index=False)
print(f"Saved with custom headers: {custom_header_filename}")

# Append mode demonstration (be careful with this)
append_filename = os.path.join(output_dir, "Append_Demo_Data.csv")
df_small = df_transformed.head(5)
df_small.to_csv(append_filename, index=False, mode='w')  # Write first
df_small.to_csv(append_filename, index=False, mode='a', header=False)  # Append without header
print(f"Demonstrated append mode: {append_filename}")

print("\nFILE SUMMARY AND VERIFICATION")

# List all created files
print("\nAll created files:")

if os.path.exists(output_dir):
    files = os.listdir(output_dir)
    total_size = 0
    
    for file in sorted(files):
        file_path = os.path.join(output_dir, file)
        if os.path.isfile(file_path):
            size = os.path.getsize(file_path)
            size_kb = size / 1024
            total_size += size
            print(f"{file:<35} {size_kb:>8.1f} KB")
    
    print(f"Total files created: {len(files)}")
    print(f"Total disk space used: {total_size/1024:.1f} KB")

print("\nREADING VERIFICATION")

# Verify we can read back all major formats
print("\nReading back saved files to verify integrity:")

test_files = [
    (os.path.join(output_dir, "New_Social_Network_Ads_Data.csv"), "CSV", pd.read_csv),
    (os.path.join(output_dir, "Social_Network_Ads_Data_records.json"), "JSON", lambda x: pd.read_json(x, lines=True)),
]

# Add Excel if available
excel_file = os.path.join(output_dir, "New_Social_Network_Ads_Data.xlsx")
if os.path.exists(excel_file):
    test_files.append((excel_file, "Excel", pd.read_excel))

# Add Parquet if available
parquet_file = os.path.join(output_dir, "Social_Network_Ads_Data.parquet")
if os.path.exists(parquet_file):
    test_files.append((parquet_file, "Parquet", pd.read_parquet))

for file_path, file_type, read_func in test_files:
    try:
        if os.path.exists(file_path):
            df_test = read_func(file_path)
            print(f"{file_type:<8} - Shape: {df_test.shape}, Columns: {len(df_test.columns)}")
        else:
            print(f"{file_type:<8} - File not found")
    except Exception as e:
        print(f"{file_type:<8} - Error: {str(e)[:50]}...")

print("\nLAB 2.4 COMPLETED SUCCESSFULLY!")

print(f"""
Summary of Operations Completed:
   CSV file saving (with/without index)
   Excel file saving (single and multi-sheet)
   JSON file saving (multiple orientations)
   Parquet file saving (if available)
   Data processing pipeline with intermediate saves
   Advanced saving options and parameters
   File verification and integrity checks

All files saved in: {output_dir}/
Dataset processed: Social Network Ads Data
Pipeline: Raw -> Cleaned -> Filtered -> Analyzed

Next steps: Explore the saved files and experiment with different parameters!
""")

Libraries imported successfully!
Pandas version: 2.3.0
NumPy version: 2.1.0
Created directory: saved_datasets

LOADING DATASET
Available datasets: ['New_Social_Network_Ads_Data.csv', 'New_Social_Network_Ads_Data.xlsx', 'Social_Network_Ads_CSV.csv', 'NoIndex_Social_Network_Ads_Data.csv', 'Social_Network_Ads.csv', 'Social_Network_Ads_JSON.json', 'Social_Network_Ads_Excel.xlsx', 'New_Social_Network_Ads_Data.json', 'Social_Network_Ads_Delimiter.csv']
Creating sample Social Network Ads dataset for demonstration...
Sample dataset created successfully!
Dataset shape: (405, 5)
Columns: ['User ID', 'Gender', 'Age', 'EstimatedSalary', 'Purchased']

Dataset Overview:
    User ID  Gender  Age  EstimatedSalary  Purchased
0  15624807    Male   58         128746.0          1
1  15624808  Female   47          53889.0          1
2  15624809  Female   34          63669.0          0
3  15624810  Female   37          72988.0          1
4  15624811    Male   42          80287.0          1

Dataset Info:
<c